In [1]:
import cv2
import numpy as np

In [3]:
image = cv2.imread('struct_image.png') # reading the image
#image = cv2.imread('Screenshot.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert2grayscale
(thresh, binary) = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) # convert2binary
#cv2.imshow('binary', binary)
#cv2.imwrite('binary.png', binary)

# find contours
contours, _ = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#image is the name of the image
#Mode is Contour retrieval mode
#Method is Contour approximation method
for contour in contours:
    #draw a rectangle around those contours on main image
    [x,y,w,h] = cv2.boundingRect(contour)
    cv2.rectangle(image, (x,y), (x+w,y+h), (0, 255, 0), 1)

In [ ]:
cv2.imshow('contour', image)
cv2.imwrite('contours.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
mask = np.ones(image.shape[:2], dtype="uint8") * 255 # create blank image of same dimension of the original image
contours, _= cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 
heights = [cv2.boundingRect(contour)[3] for contour in contours] # collecting heights of each contour
avgheight = sum(heights)/len(heights) # average height
# finding the larger contours
# Applying Height heuristic
for c in contours:
    [x,y,w,h] = cv2.boundingRect(c)
    if 2*h > avgheight:
        cv2.drawContours(mask, [c], -1, 0, -1)
# cv2.imshow('filter', mask)
cv2.imwrite('filter.png', mask)



True

In [4]:
from pythonRLSA import rlsa
import math
x, y = mask.shape
value_x = max(math.ceil(x/100),math.ceil(y/100))+20 #heuristic
value_y = max(math.ceil(x/100),math.ceil(y/100))+8 #heuristic
mask1 = rlsa.rlsa(mask, False, True, value_y) #rlsa application
mask2= rlsa.rlsa(mask, True,False, value_x) #rlsa application
# cv2.imshow('rlsah', mask)
mask=mask1&mask2


In [5]:
mask3 = np.ones((3,3), dtype="uint8") * 255 # blank 3*3 matrix
img_dilation = cv2.dilate(mask, mask3, iterations=2)
cv2.imwrite('Dilation.png', img_dilation)
print(img_dilation)

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [6]:
def dfs(x,y):
    global x_min,y_min,x_max,y_max
    if(x>0 and flag[x-1][y]==255 and img_dilation[x-1][y]==0):
        flag[x-1][y]=0;
        x_min=min(x_min,x-1)
        dfs(x-1,y)
        
    if(x<r-1 and flag[x+1][y]==255 and img_dilation[x+1][y]==0):
        flag[x+1][y]=0;
        x_max=max(x_max,x+1)
        dfs(x+1,y)
        
    if(y>0 and flag[x][y-1]==255 and img_dilation[x][y-1]==0):
        flag[x][y-1]=0;
        y_min=min(y_min,y-1)
        dfs(x,y-1)
        
    if(y<c-1 and flag[x][y+1]==255 and img_dilation[x][y+1]==0):
        flag[x][y+1]=0;
        y_max=max(y_max,y+1)
        dfs(x,y+1)  
    return

In [ ]:
global flag
flag= np.ones(gray.shape[:2], dtype="uint8") * 255 
li=[]
r=image.shape[0]
c=image.shape[1]
global x_min,y_min,x_max,y_max
for i in range(0,r):
    
    for j in range(0,c):
        if(flag[i][j]!=0 and img_dilation[i][j]==0):
            print (i,j)
            x=i
            y=j
            flag[x][y]=0
            x_min=x
            x_max=x
            y_min=y
            y_max=y
            dfs(x,y)
            w=x_max-x_min
            h=y_max-y_min
            li.append([x,y,w,h])
            print(li)
            print('yes')
            
print(li)

27 274
[[27, 274, 16, 197]]
yes
96 192


In [6]:
cv2.rectangle(image, (192,96), (205,105), (255, 255, 0), 1)
cv2.imwrite('bb.png', image)

True

In [76]:
contours, _ = cv2.findContours(~mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # find contours
mask2 = np.ones(image.shape, dtype="uint8") * 255 # blank 3 layer image
for contour in contours:
    [x, y, w, h] = cv2.boundingRect(contour)
    if w > 0.60*image.shape[1]: # width heuristic applied
        title = image[y: y+h, x: x+w] 
        mask2[y: y+h, x: x+w] = title # copied title contour onto the blank image
        image[y: y+h, x: x+w] = 255 # nullified the title contour on original image
cv2.imshow('title', mask2)
cv2.imwrite('title.png', mask2)
cv2.imshow('content', image)
cv2.imwrite('content.png', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [77]:
import pytesseract
try:
    from PIL import Image
except ImportError:
    import Image
pytesseract.pytesseract.tesseract_cmd = r'F:\Program Files\Tesseract-OCR\tesseract'
title = pytesseract.image_to_string(Image.fromarray(mask2))
content = pytesseract.image_to_string(Image.fromarray(image))
print(title)
print(content)

Every day, peopl aroiind the worldluse maps. WWhefhen if is an girplaue pilot on Businessman, ous wif
on museum group, caps have always and will continus to providle usefial infixsatiom fim all

Mapping ths World falkis about this uses offmaps, as welll as how fo Giffdrentiats babkean th Eps ofraap
projection andl fjps ofimap,
nlthis series, we ftavel f0 thes past and Learn about historical mapmakers, torn Olbiadius Ptolkxny (ia

 

 

 

tmvolor proton would be used in uaz ways. We coal se it im population concent,

 

bighiways, Land alewation, and so ony ings!
Eon example, swe coil mike wtopographie map ofthis U.S, hich stn ll elevation. We oid ake

  
   

 

 

 

 

it could even be amp thai Shows thi 90 states, thir capitals anil borékens|
excursion is fhe usan Fal, wbiere wwe sexe sous Hypothetical solutions
Cumently, wwe are working on batter virtual map technology,

(Nowy, scientists baive Beer abls fo put maps on phones. Buck in this early U900s, peoples bail ta lug alot of


In [36]:
"""But the pictures, lines that exist in the image might cause the issue.
If the pictures exist close to the title might connect when we perform Vertical RLSA. So clean the image before.

To avoid the complications, remove the pictures and lines present in the image. 
Pictures can be removed by the area, width, height of the contour. Pictures surely will be the largest contours of the image.
Lines can be detected and separated out by Canny edge detection and Houghlines algorithms.

Detect lines and pictures and draw over another binary image and do bitwise-and with the original image to subtract out noise.
"""

minLineLength = 100
maxLineGap = 50
def lines_extraction(gray):
    #this function extracts the lines from the binary image. Cleaning process.
    edges = cv2.Canny(gray, 75, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength, maxLineGap)
    return lines
mask = np.ones(image.shape[:2], dtype="uint8") * 255 # create a white image
#print(gray.size)
lines = lines_extraction(gray) # extract lines
try:
    for line in lines: # write lines to mask
        x1, y1, x2, y2 = line[0]
        cv2.line(mask, (x1, y1), (x2, y2), (0, 255, 0), 3)
except TypeError:
    pass

contours, _ = cv2.findContours(~binary,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) # find contours
areas = [cv2.contourArea(c) for c in contours] # find area of contour
avgArea = sum(areas)/len(areas) # finding average area
for c in contours:# average area heuristics
    if cv2.contourArea(c)>60*avgArea:
        cv2.drawContours(mask, [c], -1, 0, -1)
binary = cv2.bitwise_and(binary, binary, mask=mask) # subtracting the noise
cv2.imwrite('noise.png', mask)
cv2.imshow('mask', mask)
cv2.imshow('binary_noise_removal', ~binary)
cv2.imwrite('binary_noise_removal.png', ~binary)
cv2.waitKey(0)
cv2.destroyAllWindows()



568480


In [89]:
flag.size

568480

In [86]:
r

760

In [87]:
c

748

In [10]:
img_dilation[250][100]

0

In [7]:
flag = np.ones(gray.shape[:2], dtype="uint8") * 255 
flag[759][747]

255